In [4]:
import cvxpy as cp
import numpy as np
from itertools import product

In [5]:
shipping_cost = np.array([[1,3,2],[3,2,2]]).reshape(2,3)
all_demand = np.array([[100,150,120],[120,180,150],[150,200,180]]).reshape(3,3)
selling_price = np.array([[16,16]]).reshape(2,1)
produce = np.array([100,200]).reshape(2,1)
production_cost = np.array([11,10]).reshape(2,1)

In [6]:
senario = np.array([[100,120,150],
       [150,180,200],[120,150,180]])
senario_comb = []
index = np.arange(0,3)
index_permute = product(index,repeat=3)
for i in index_permute:
    temp_array =[]
    for j in range(0,3):
        temp_array.append(float(senario[j,:][i[j]]))
    senario_comb.append(temp_array)
all_demand = np.array(senario_comb).reshape(27,3)

In [31]:
produce_over_scenario = []
objective_over_scenario = []
for demand in all_demand:
    produce = cp.Variable(shape=(2,1))
    sell = cp.Variable(shape=(2,3))
    salvage = cp.Variable(shape=(2,1))
    produce_limit = np.array([300,300]).reshape(2,1)

    constraints_senario = []
    constraints_senario.append(cp.sum(sell,axis=1,keepdims=True) +salvage <= produce)
    constraints_senario.append(cp.sum(sell,axis=0,keepdims=True) <= demand.reshape(1,-1))
        
    
    constraints = [produce>=0,produce<=produce_limit,sell>= np.zeros_like(sell),salvage>= np.zeros_like(salvage)]
    constraints.extend(constraints_senario)


    objective = cp.Minimize((production_cost.T@produce)-(16)*cp.sum(sell,keepdims=True) + (1)*cp.trace(((((np.repeat([shipping_cost],1,axis=0).reshape(-1,3))@(sell.T))))))
    problem = cp.Problem(objective, constraints)
    problem.solve()
    produce_over_scenario.append(produce.value)
    objective_over_scenario.append(problem.value)

In [16]:
produce_over_scenario = np.array(produce_over_scenario).reshape(27,2)

In [32]:
objective_over_scenario = np.array(objective_over_scenario).reshape(27,1)

In [33]:
value_of_imprfction = -1609.9999997418313 - np.mean(objective_over_scenario)

159.9999967386409
